This code implements a simulation software for the S-ICEP problem.


In [1]:
import numpy as np
import random as rm
import math
import pandas as pd
from scipy.stats import uniform
from matplotlib import pyplot as plt
from statsmodels.distributions.empirical_distribution import ECDF
from getStats import getStats
from earthQuake import EarthQuake
from flood import Flood
from population import Population
from weather import Weather
from scenario import Scenario
from utils import Utils
import random

We as first set initial parameters

In [5]:
numScenarios = 25
T = 365 *1              # Select number of years
basePop = 1000
period = 100
T = 360*100
#weather = Weather(T)
weather = Weather(T)
simulation = []
dailyRain, t_min, t_max, radiaz = weather.weatherGame()
dailyStats = []
oneDayRain = []
oneDayt_min = []
oneDayt_max = []
oneDayradiaz = []
stats = []
for d in range(360):
    for t in range(period):        
        oneDayRain.append(dailyRain[d + 360*t])
        oneDayt_min.append(t_min[d + 360*t])
        oneDayt_max.append(t_max[d + 360*t])
        oneDayradiaz.append(radiaz[d + 360*t])
    rainDesc = np.mean(oneDayRain), np.var(oneDayRain)
    t_minDesc = np.mean(oneDayt_min), np.var(oneDayt_min)
    t_maxDesc = np.mean(oneDayt_max), np.var(oneDayt_max)
    radiationDesc =  np.mean(oneDayradiaz), np.var(oneDayradiaz)
    dailyStats.append((rainDesc, t_minDesc, t_maxDesc, radiationDesc))


{p01: 5.89888273533717e-6, p11: 0.994556307602081}


Let'run our simulation;
The first stage of the process consists of symulating wet/dry days by means of a 2-states markov process: dry days or wet days.
Each trasnsition is associated to a transition probability, then we have: 
$$M = \begin{pmatrix}
p_{00} & p_{01} \\
 p_{10} & p_{11}
\end{pmatrix}$$

$$p_{01}:  \text{probability of wet day after dry day} \\
p_{11}:  \text{probability of wet day after wetday} \\
p_{00} = 1 -  p_{01} \\
p_{10} = 1 -  p_{11}
$$


Each "weather game" iterates as follows:
![weather game](./images/weather-game.png "Title")





For each day model "decides" wether it is rainy or not, if yes then calculates the effectively rain amount by using an exponential distribution of mean u $$f(x) = (1/\mu ) \mathrm{e}_{}^{-x/\mu}$$
if no, rain amount is zero.

In [ ]:
weatherData = weather.weatherGame()


Next we proceed to sample non precipitational variable, that are:
<ul>
<li> Min temperatures </li>
<li> Max temperatures </li>
<li> Solar radiation </li>
</ul>
Note:
Those quantities, are poorly considered for the natural disaster triggering, but are usefull to compute severity parameters during an evacuation, affecting driving and operation times, visibility etc.

Quantities, (t_min, t_max, rad) are extracted from a stochastic Auto Regressive first-order process, defined by:
$$z(t) = Az(t-1) + B \epsilon(t) $$
where 
$$ z(t) =  \begin{pmatrix}
\mathrm{z}_{t_m}^{} \\
\mathrm{z}_{t_M}^{} \\
 \mathrm{z}_{rad}^{}\\

\end{pmatrix}

\\\epsilon(t) \sim G(0, 1)
$$ 
and 

$$
A = \begin{pmatrix}
0.567 & 0.086 & -0.002 \\
0.253 & 0.504 & -0.05 \\
-0.006 & -0.039 & 0.244
\end{pmatrix}

B = \begin{pmatrix}
0.781& 0 & 0 \\
0.328 & 0.637 & 0 \\
0.238 & -0.341 & 0.873
\end{pmatrix}
$$

Note: A and B are given from [PAPER]


Once they have been generated, the z(t) are transformed to weather
variables in a way that depends on whether the day has been simulated to be wet or
dry
$$\mathrm{T}_{k}^{}(t) = \begin{cases} \mathrm{\mu}_{k,0}^{}(t) + \mathrm{\sigma}_{k,0}^{}(t) \mathrm{z}_{k}^{}(t) & \text{if is a dry day}
\\ \mathrm{\mu}_{k,1}^{}(t) + \mathrm{\sigma}_{k,1}^{}(t) \mathrm{z}_{k}^{}(t) & \text{if is a wet day}
\end{cases}$$

Where seasonal fluctuation are considered by imposing to means a sinusoidal time-dependent behaviour, indeed:
$$\mu(t) = \overline{\mu} + C cos (0.0172(t -T))$$


In [ ]:
dailyRain, t_min, t_max, radiaz = weatherData


Let's move on population data generation:
Since the of population follows a cyclical behaviour during the year it is reasonable to represent it with one (or more) sinusoids, plus some uncertainty extracted from a normal distibution
$$
population(t) = \overline{pop} * (1 + \left[ Acos(0.0172t) + Bcos(0.0172t +\Phi) \right] + C \epsilon(t))
\\
\epsilon(t) \sim G(0, 1)
$$

The resulting process is then smoothed with a moving avarage filter with a box window of 7 days in order to smooth population variations.

In [ ]:
population = Population(basePop, T)


In [ ]:
Ia = 10
S = 11
Q = [(((p - Ia)**2)/(p - Ia + S)) for p in dailyRain[0:T]]

Then we can plot results:

In [ ]:

fig1, axs1 = plt.subplots(3, 2)

X = range(0, T)

axs1[0][0].plot(X, oneDayRain[0:T] )
axs1[0][0].set_title('Rainy days')

axs1[1][0].plot(X, oneDayt_min, oneDayt_max)
axs1[1][0].set_title('t max/min')

axs1[0][1].plot(X,  population.getPopWave() )
axs1[0][1].set_title('pop')

axs1[1][1].plot(X,  oneDayradiaz )
axs1[1][1].set_title('radiation')

axs1[2][0].plot(X,  Q )
axs1[2][0].set_title('rainOff')

plt.show()


Now, we need to collect data by letting run our model for a useful period, let's say 100 years.

Let's move on now to sampling natural disasters, we start from a random sampled day, compute the corresponding weather variables and evaluate the probabilty of the emergency:
Two types of disaster are considered: 
<ul>
<li> Earthqwakes: following the <a  href="https://en.wikipedia.org/wiki/Gutenberg%E2%80%93Richter_law"  > Gutenberg-Richter distribution </a>


</ul>

$$
N = \mathrm{10}_{}^{a-bM}
$$
Where N is the total number of Earthquake of magnitude M or above, a and b are parameters to be chosen accordingly to geo-activity of the area of interest.
<ul>

<li> Floods: following a <a href="https://en.wikipedia.org/wiki/100-year_flood#:~:text=A%20100%2Dyear%20flood%20is,exceeded%20in%20any%20given%20year."> 100-year flood probability </a> 
</li>

</ul>

$$
P = 1 - [1 - (1/T)]^{n}
$$
Where P is the probability of one or more floods occurring during any period will exceed a given flood threshold. T is the return period, expressed as the inverse of the average frequency of occurrence (in that case T = 10).

At this point, we can generate natural disasters each with an associated probability.
As first step, disaster type is extracted from a uniform distribution. 2 emergencies are possible:
<ul>
<li>In case of earthquake the day T is randomly extracted from uniform distribution </li>
<li>In case of floods, first the most rainy Days are ranked, then T is randomly extracted from this subset </li> 
</ul>



In [ ]:
scenarios = []
tIstants = []
mostRainyWeeks = weather.setMostRainyWeeks(numScenarios)

for ns in range(numScenarios):
    disasterType = random.randint(1, 2)
    if disasterType == 1:
        timeIstant = random.randint(0, T)
    else:
        index = random.sample(range(len(mostRainyWeeks)), 1)[0]
        sample = mostRainyWeeks[index]
        mostRainyWeeks = np.delete(mostRainyWeeks, index)
        timeIstant = int(sample)
    scenario = Scenario(population, weather, numScenarios, timeIstant, disasterType)    
    s = scenario.sampleScenario()


    scenarios.append(s)
    tIstants.append(s.timeIstant)


The final scenario's probability is given by the probability of the natural disaster, times the probability relative to population and temperatures deviation from means; which are are upper bounded with the <a href="https://en.wikipedia.org/wiki/Cantelli%27s_inequality"> Cantelli's inequality</a>

Once every scenario has his absolute probability assigned, finally we normalize all the probabilities in order to make them sum to 1.

In [ ]:
X = range(0, T)
df = pd.DataFrame(columns=['day', 'population', 'rainAmount', 't_min', 't_max', 'radiation', 'disaster', 'Relative Probability'])

df['day'] = tIstants

df['population'] = [int(s.population) for s in scenarios]
df['rainAmount'] = [int(s.rainAmount) for s in scenarios]
df['t_min'] = [int(s.tempMin) for s in scenarios]
df['t_max'] = [int(s.tempMax) for s in scenarios]
df['radiation'] = [int(s.radiation) for s in scenarios]
# df['flood'] = [s.flooding[0] for s in scenarios]
# df['earthqwake'] =  [s.earthqwake[0] for s in scenarios]
# df['probability'] = [s.probability for s in scenarios]
df['Relative Probability'] = Utils.normalizeProbs([s.probability for s in scenarios])
disasterTypes = []
for s in scenarios:
    dis = "Flood" if s.flooding[0] > 0 else "Earthquake, Mag: "+str(s.earthqwake[0])
    dis = "Flood" + "Earthquake, Mag: "+str(s.earthqwake[0]) if (s.flooding[0] > 0 and s.earthqwake[0]) > 0 else dis
    disasterTypes.append(dis)
df['disaster'] = disasterTypes

df.sort_values(by=['day'], inplace=True)

df
